# Underlaying Event uncertainty

In [1]:
import os, sys, time
sys.path.append(os.getcwd() + "/../")
sys.path.append(os.getcwd() + "/../tools/")
sys.path.append(os.getcwd() + "/../scripts/uncertainties_dir/")

from prep_ana_II import *
from cross_section_calc import calc_cross_stuff, cross_calc, stat_unc_calc, normalization_unc_calc, pseudo_data_yield_sum
from jet_scale import jec_setup, jet_scale_shift_flat, pseudo_data_yield_sum
from sklearn.externals import joblib

Welcome to JupyROOT 6.10/00
Loading binning options...
Loading plotting options
unc_mc_process and scales are parameter dictionaries


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 

In [3]:
#ToDo:
#+?Determine why both up and down variaitions have a larger cross-section
#jer_obj = jec_setup(unc="lep")
#jet_scale_shift_flat(jer_obj.df, jet_pt_shift=1., rf=jer_obj.rfs)
#df = pre_cuts(jer_obj.df, diff_charge= False)
#df_da = jer_obj.df_da
#jet_scale_shift_flat(jer_obj.df_da, jet_pt_shift=1., rf=jer_obj.rfs)
#df_ww = jer_obj.df_ww

df = load_testset(unc="")
df_da = load_presel_w_fDY_fTT_DATA(unc="")
df_ww = rp.read_root(data_path+"/ww_complete.root", columns=columns)

Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo of class string read from file /home/gunter/WW_analysis/data_alpha/ww_complete.root
   has the same version (=2) as the active class but a different checksum.
   You should update the version to ClassDef(string,3).
   Do not try to write objects with the current class definition,
   the files will not be readable.

Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::_Alloc_hider _M_dataplus; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   unsigned long _M_string_length; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::(anonymous) ; //


In [4]:
def score_df(df, rfs):
    
    features_fDY = rfs["features_fDY"]
    features_fTT = rfs["features_fTT"]
    clf_fDY = rfs["clf_fDY"]
    clf_fTT = rfs["clf_fTT"]
    
    #Remove undesirable measurements TT
    temp = df[features_fTT]
    temp = temp.replace([np.inf, -np.inf], 0)

    #Predict TT
    pred_fTT = clf_fTT.predict_proba(np.float32(temp.values))
    df["pred_fTT_WW"] = pred_fTT[:,0]

    #Remove undesirable measurements DY
    temp = df[features_fDY]
    temp = temp.replace([np.inf, -np.inf], 0)
    pred_fDY = clf_fDY.predict_proba(np.float32(temp.values))
    df["pred_fDY_WW"] = pred_fDY[:,0]

In [5]:
def yield_string(df, string):
    raw_numbers = {}
    for process in df.process.unique():
        j0 = 0
        j1 = 0
        j2 = 0
        for process_decay in df[df.process == process].process_decay.unique():
            j0 += jer_obj.rf_ana(df[(df.process_decay == process_decay) & (df.numb_jets == 0) & (df.lep1_Charge != df.lep2_Charge)], "pred_fDY_WW", "pred_fTT_WW").weight.sum() * scales[process_decay]
            j1 += jer_obj.rf_ana(df[(df.process_decay == process_decay) & (df.numb_jets == 1) & (df.lep1_Charge != df.lep2_Charge)], "pred_fDY_WW", "pred_fTT_WW").weight.sum() * scales[process_decay]
            j2 += jer_obj.rf_ana(df[(df.process_decay == process_decay) & (df.numb_jets == 2) & (df.lep1_Charge != df.lep2_Charge)],"pred_fDY_WW", "pred_fTT_WW" ).weight.sum() * scales[process_decay]


        raw_numbers[process] = np.array([j0,j1,j2], dtype=float)
        string += process + "\t" +  str(j0) + "\t" + \
                                    str(j1) + "\t" + \
                                    str(j2) + "\n"
    return string, raw_numbers

In [6]:
#Unpack Random forest 
random_forests = joblib.load("../data/RF/dec_05_fDY_fTT.jbl")
features_fDY = random_forests["features_fDY"]
clf_fDY = random_forests["clf_fDY"]
features_fTT = random_forests["features_fTT"]
clf_fTT = random_forests["clf_fTT"]

In [7]:
#WW up and down data frames
df_ww_up = rp.read_root(  data_path+"/ww_up_complete.root",   columns=columns_lep_unc)
df_ww_down = rp.read_root(data_path+"/ww_down_complete.root", columns=columns_lep_unc)

df_ww_up["process"] = "WW"
df_ww_down["process"] = "WW"
df_ww_up["process_decay"] = "WW"
df_ww_down["process_decay"] = "WW"

df_ww_up["gen_weight"] = df_ww_up.gen_weight.values/ np.abs(df_ww_up.gen_weight.values)
df_ww_up["weight"] = df_ww_up.weight.values * df_ww_up.gen_weight.values

df_ww_down["gen_weight"] = df_ww_down.gen_weight.values/ np.abs(df_ww_down.gen_weight.values)
df_ww_down["weight"] = df_ww_down.weight.values * df_ww_down.gen_weight.values

score_df(df_ww_down, random_forests)
score_df(df_ww_up, random_forests)


In [8]:
#produce pseudo data
pseudo = pseudo_data_yield_sum(df, df_da)#This was the problem whtis was why you were getting different values
cross_orig   =  cross_calc(df, df_da, df_ww, scales,  fiducial=True, pseudo=pseudo)

cross_sections = {}
print df.shape

for it, ww_variant in enumerate([df_ww_down, df_ww_up]):
    df = df[df.process != "WW"]
    df = pre_cuts(pd.concat([df, ww_variant]))
    
    ww_type = ""
    if it == 0:
        ww_type = "down"
        scales["WW"] = 35.9e3 * 118.7 * (3*.108)**2 / 1987956.
    else:
        ww_type = "up"
        scales["WW"] = 35.9e3 * 118.7 * (3*.108)**2 / 1866452.
    
    cross_sections[ww_type] = cross_calc(df, df_da, ww_variant, scales,  fiducial=True, pseudo=pseudo)
    
    print ww_type, df[(df.process == "WW") & (df.pred_fDY_WW > .9) & (df.pred_fTT_WW > .6)].shape[0] / float(ww_variant.shape[0])
    print df.shape

(5293944, 46)
down 0.143905201413
(4859509, 50)
up 0.141596851527
(4841916, 50)


In [9]:
print cross_sections

{'down': 2.1252523374847097, 'up': 2.0222495398114377}


In [10]:
cross_orig

1.6894087625897014

In [11]:
1.566/1.637

0.9566279780085523

# RF test

<pre>
Nominal Training

  Down                Up            Nominal 
8869 1593         8759 1563       8846  1540
</pre>

In [13]:
df_ww.process.unique()

array(['WW'], dtype=object)